# Testing MFCC features

## Importing Libraries

In [2]:
from sklearn.mixture import GaussianMixture
import pandas as pd
import numpy as np
import math
import pickle
from sklearn.mixture.gaussian_mixture import _compute_precision_cholesky

## Loading test data

In [4]:

def transform(file1,file2):
    x=np.load(file1).tolist()
    y=np.load(file2).tolist()
    return pd.DataFrame({"feature":x,"label":y})


pathf=["features/mfcc_test/features_test.npy","features/mfcc_test_delta/features_test_delta.npy","features/mfcc_test_delta_delta/features_test_delta_delta.npy"]
pathl=["features/mfcc_test/labels_test.npy","features/mfcc_test_delta/labels_test_delta.npy","features/mfcc_test_delta_delta/labels_test_delta_delta.npy"]
df = [transform(pathf[0],pathl[0]),transform(pathf[1],pathl[1]),transform(pathf[2],pathl[2])]



## Computing Likelihood 

In [ ]:

delta_with=["./models/mfcc/with_energy_coeff/","./models/mfcc_delta/with_energy_coeff/","./models/mfcc_delta_delta/with_energy_coeff/"]
delta_without=["./models/mfcc/without_energy_coeff/","./models/mfcc_delta/without_energy_coeff/","./models/mfcc_delta_delta/without_energy_coeff/"]
dl=[[0],[0,14],[0,14,27]]

ph_score_with=[]
ph_score_without=[]
ph_score_comp=[]

for j in range(0,3):
    print(j)
    
    ph_score_with.append([])
    ph_score_without.append([])
    ph_score_comp.append([])
    phoneme=df[j]['label'].unique()
    phoneme.sort()
    
    for ph in phoneme:
        print(ph)
        temp=df[j]
        features=np.array(temp["feature"].tolist())
        #labels.append(np.array(temp["label"].tolist()))
        
        models=pickle.load(open(delta_with[j]+'/64/'+ph+".pkl", 'rb'))
        ph_score_with[j].append(models.score_samples(features).tolist())
        
        features=np.delete(features,dl[j], axis=1)
        
        models=pickle.load(open(delta_without[j]+'/64/'+ph+".pkl", 'rb'))
        ph_score_without[j].append(models.score_samples(features).tolist())
       
        
        if j==0:
            
            comp=[2,4,8,16,32,128,256]
            ph_score_comp=np.empty((7),dtype=[])
            
            ph_score_comp=[]
            for i in comp:
                ph_score_comp.append([])
            
            
            count=0
            for i in comp:
                
                print(i)
                
                models=pickle.load(open(delta_without[j]+'/'+(str)(i)+"/"+ph+".pkl", 'rb'))
                ph_score_comp[count].append(models.score_samples(features).tolist())
                count=count+1
                
        
                
                
        
        
        

## Making phoneme dictionary

In [6]:

phoneme=df[0]['label'].unique()
phoneme.sort()

ph={v: k for k, v in dict(enumerate(phoneme)).items()}

print(ph)

{'': 0, 'aa': 1, 'ae': 2, 'ah': 3, 'aw': 4, 'ay': 5, 'b': 6, 'ch': 7, 'd': 8, 'dh': 9, 'dx': 10, 'eh': 11, 'er': 12, 'ey': 13, 'f': 14, 'g': 15, 'hh': 16, 'ih': 17, 'iy': 18, 'jh': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'ng': 24, 'ow': 25, 'oy': 26, 'p': 27, 'r': 28, 's': 29, 'sh': 30, 'sil': 31, 't': 32, 'th': 33, 'uh': 34, 'uw': 35, 'v': 36, 'w': 37, 'y': 38, 'z': 39}


## Encoding phoneme to numbers

In [7]:
labels=np.array(df[0]["label"].tolist())
labels=np.array(list(map(lambda x:ph[x],labels)))

## Computing Accuracies

In [8]:
delta=["no_delta","delta+1","delta+2"]
for j in range(0,3):
    predicted=np.argmax(ph_score_with[j],axis=0).reshape(-1)
    accuracy=np.where(predicted==labels)[0].shape[0]/labels.shape[0]
    print("component: 64"," ",delta[j] ," with_energy_coefficient "," accuracy: ",accuracy)
    
    predicted=np.argmax(ph_score_without[j],axis=0).reshape(-1)
    accuracy=np.where(predicted==labels)[0].shape[0]/labels.shape[0]
    print("component: 64"," ",delta[j] ," without_energy_coefficient "," accuracy: ",accuracy)
    
comp=[2,4,8,16,32,128,256]
for i in range(len(comp)):
    predicted=np.argmax(ph_score_comp[i],axis=0).reshape(-1)
    accuracy=np.where(predicted==labels)[0].shape[0]/labels.shape[0]
    print("component: ",comp[i]," no_delta "," without_energy_coefficient "," accuracy: ",accuracy)
    

component: 64   no_delta  with_energy_coefficient   accuracy:  0.14409954390470708
component: 64   no_delta  without_energy_coefficient   accuracy:  0.1329318513926405
component: 64   delta+1  with_energy_coefficient   accuracy:  0.18861090200593367
component: 64   delta+1  without_energy_coefficient   accuracy:  0.17534207146968958
component: 64   delta+2  with_energy_coefficient   accuracy:  0.19261391312048887
component: 64   delta+2  without_energy_coefficient   accuracy:  0.17387636717885135
component:  2  no_delta   without_energy_coefficient   accuracy:  0.024009210468051188
component:  4  no_delta   without_energy_coefficient   accuracy:  0.024009210468051188
component:  8  no_delta   without_energy_coefficient   accuracy:  0.024009210468051188
component:  16  no_delta   without_energy_coefficient   accuracy:  0.024009210468051188
component:  32  no_delta   without_energy_coefficient   accuracy:  0.024009210468051188
component:  128  no_delta   without_energy_coefficient   accu

In [ ]:
feat=[np.load("features_test.npy"),np.load("features_test_delta.npy"),np.load("features_test_delta_delta.npy")]
lab=[np.load("labels_test.npy"),np.load("labels_test_delta.npy"),np.load("labels_test_delta_delta.npy")]

## Computing ground truth and predicted 

In [ ]:
delta_with=["D:\deeplearning\Examples\ASR_2018_T01-master\p1\TEST_PER/mfcc/with_energy_coeff/","./TEST_PER/mfcc_delta/with_energy_coeff/","./TEST_PER/mfcc_delta_delta/with_energy_coeff/"]
delta_without=["D:\deeplearning\Examples\ASR_2018_T01-master\p1\TEST_PER/mfcc/without_energy_coeff/","./TEST_PER/mfcc_delta/without_energy_coeff/","./TEST_PER/mfcc_delta_delta/without_energy_coeff/"]

sentences_with=[]
sentences_without=[]
sentences_comp=[]
for j in range(3):
    sentences_with.append("")
    sentences_without.append("")
for i in range(7):
    sentences_comp.append("")

sentences_ground=""
sm=0
for i in range(feat[j].shape[0]):
    print(i)
        
    for j in range(0,3):
        
        
        
        tp=np.array(ph_score_with[j])
        
        sentences_with[j]= sentences_with[j]+np.array2string(np.argmax(tp[:,sm:sm+feat[j][i].shape[0]],axis=0),max_line_width=2000)+"\r\n"
        
        tp=np.array(ph_score_without[j])
        sentences_without[j]=sentences_without[j]+np.array2string(np.argmax(tp[:,sm:sm+feat[j][i].shape[0]],axis=0),max_line_width=2000)+"\r\n"
        
    
    sentences_ground=sentences_ground+np.array2string(labels[sm:sm+feat[j][i].shape[0]],max_line_width=30000)+"\n"
      
    for j in range(7):
        tp=np.array(ph_score_comp[j])
        sentences_comp[j]=sentences_comp[j]+np.array2string(np.argmax(tp[:,sm:sm+feat[0][i].shape[0]],axis=0),max_line_width=2000)+"\r\n"
        
    sm=sm+feat[0][i].shape[0]
            
        #print(feat[j][i].shape[0])
        #input()
    

## Dumping ground truth and predicted on TEST PER

In [ ]:
path="D:/deeplearning/Examples/ASR_2018_T01-master/p1/"
delta_with=[path+"TEST_PER/mfcc/with_energy_coeff/",path+"TEST_PER/mfcc_delta/with_energy_coeff/",path+"TEST_PER/mfcc_delta_delta/with_energy_coeff/"]
delta_without=[path+"TEST_PER/mfcc/without_energy_coeff/",path+"TEST_PER/mfcc_delta/without_energy_coeff/",path+"/TEST_PER/mfcc_delta_delta/without_energy_coeff/"]

with open(path+"TEST_PER/mfcc_ground/label.txt","w") as f:
    f.write(str(sentences_ground))

for j in range(3):
    
    f=open(delta_with[j]+"_64.txt","w")
    f.write(sentences_with[j])
    f.close()
    
    f=open(delta_without[j]+"_64.txt","w")
    f.write(sentences_without[j])
    f.close()
    
 
    
comp=[2,4,8,16,32,128,256]
for i in range(7):
    f=open(delta_without[0]+"_"+str(comp[i])+".txt","w")
    f.write(sentences_comp[i]+"")
    f.close()
    
    
    
    
    